# **Sarcasm detection on Reddit**

In this notebook, we addressed a binary classification problem aimed at detecting sarcasm in Reddit comments.

We chose 3 feature engineering methods and 4 classification models and compared them to find out which combination performs better:
- Feature engineering: TF-IDF, Word2Vec, and BERT.

- Classification models: Logistic regression, Random forest, Linear SVC, and Multilayer perceptron.

## **Spark Setup**

### 1. Install PySpark and related dependencies

In [ ]:
# java
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz
# !tar xf spark-3.2.4-bin-hadoop2.7.tgz
# !rm -rf /content/spark-3.2.4-bin-hadoop2.7.tgz
!pip install findspark

In [ ]:
# !update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

In [ ]:
# setting java path as environment variable
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
!pip install pyspark
!pip install spark-nlp==4.4.2

In [ ]:
!python --version

In [ ]:
!pyspark --version

### 2. Import useful Python packages

In [ ]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import DataFrame
from pyspark.ml.feature import VectorAssembler, SQLTransformer, Normalizer
from pyspark.sql.functions import udf, col, lower, trim, regexp_replace, transform

import sparknlp

import gc

### 3. Create Spark context

In [ ]:
import findspark
findspark.init()

In [ ]:
spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[4]") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.executor.memoryOverhead", "1g") \
    .config("spark.driver.memory","32G") \
    .config("spark.python.worker.memory","32G") \
    .config("spark.sql.analyzer.maxIterations", "6000") \
    .config("spark.driver.cores", "10") \
    .config("spark.driver.maxResultSize", "32G") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.4.2") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

In [ ]:
sc = spark.sparkContext
type(sc)

## **Dataset**

The original dataset used for this task is available [here](https://www.kaggle.com/datasets/danofer/sarcasm).
This dataset contains 1.3 million Sarcastic comments from the Internet commentary website Reddit.
The dataset was generated by scraping comments from Reddit containing the `\s` (sarcasm) tag. This tag is often used by Redditors to indicate that their comment is in jest and not meant to be taken seriously, and is generally a reliable indicator of sarcastic comment content.

### Download dataset from Kaggle

In order to get the dataset from Kaggle library, a kaggle.json file is required. It can be downloaded from our personal account page and uploaded in the same folder of this notebook.

In [ ]:
# Install the Kaggle library
! pip install kaggle

# Make a directory named “.kaggle”
! mkdir kaggle

# Copy the “kaggle.json” into this new directory
! cp kaggle.json kaggle/

# Allocate the required permission for this file.
! chmod 600 kaggle/kaggle.json

# Download the dataset
! kaggle datasets download danofer/sarcasm -f train-balanced-sarcasm.csv

# Unzip the directory
! unzip train-balanced-sarcasm.csv.zip

# Delete zip file
! rm -rf train-balanced-sarcasm.csv.zip

# Make a directory to save final models
! mkdir models

### Data loading

Due to a limited amount of resources at our disposal, in order to use the *BERT Embedding* we were forced to work with only part of the original dataset. Therefore, we randomly selected only 20% of the comments in the dataset.

In [ ]:
def load_dataset_partitioned(dataset_file_path = "train-balanced-sarcasm.csv", partitioned = False):

    df_from_csv = spark.read.csv("train-balanced-sarcasm.csv", header=True)

    print("Dataframe size (n. of rows): {:d}\n".format(df_from_csv.count()))
    print("Dataframe schema:")
    df_from_csv.printSchema()
    df_from_csv.show(10)

    # selecting only the columns we are interested in
    df = df_from_csv.drop("author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment")

    if partitioned:
        # selecting 20% of the dataset
        df, second_partition = df.randomSplit([0.1, 0.9], seed = 42)
        second_partition.unpersist()
        del second_partition

    print("\nSelecting only the columns we are interested in")
    print("Dataframe with comment length size (n. of rows): {:d}\n".format(df.count()))
    print("Dataframe schema:")
    df.printSchema()
    df.show(10)

    print("Number of NULL entry/ies: {:d}".format(df.where(col("comment").isNull()).count()))
    # remove NULL entry/ies
    df = df.na.drop(subset=["comment"])

    print("Number of NULL labels: {:d}\n".format(df.where(col("label").isNull()).count()))
    # remove NULL labels
    df = df.na.drop(subset=["label"])

    print("Let's verify our dataset is still balanced:")
    df.groupBy('label').count().show()

    return df

### Data Pre-processing

Starting from the raw text data, we apply a standard NLP preprocessing pipeline.

In [ ]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, col

In [ ]:
def clean_text(df, column_name="comment"):

    # Text preprocessing pipeline
    print("***** Text Preprocessing Pipeline *****\n")

    # convert the text into a Spark-NLP annotator-ready form
    documentAssembler = DocumentAssembler() \
     .setInputCol(column_name) \
     .setOutputCol('document')

    tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('token')

    normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized") \
    .setLowercase(True) \
    .setCleanupPatterns(["""[^\w\d\s]"""]) # remove punctuations (keep alphanumeric chars)

    # note that lemmatizer needs a dictionary. So we used the pre-trained model (note that it defaults to english)
    lemmatizer = LemmatizerModel.pretrained("lemma_antbnc") \
     .setInputCols(['normalized']) \
     .setOutputCol('lemma')

    stopwords_cleaner = StopWordsCleaner.pretrained("stopwords_en") \
     .setInputCols(['lemma']) \
     .setOutputCol('clean_lemma') \
     .setCaseSensitive(False)

    # convert tokens back to human-readable form
    finisher = Finisher() \
     .setInputCols(['clean_lemma']) \
     .setOutputCols("finisher_result") \
     .setCleanAnnotations(False) \
     .setOutputAsArray(True)

    pipeline = Pipeline() \
     .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,
        stopwords_cleaner,
        finisher
     ])

    cleaned_df = pipeline.fit(df).transform(df).select(['comment', 'document', 'clean_lemma', 'finisher_result', 'label'])

    #########################################

    cleaned_df = cleaned_df.filter("finisher_result != array() and finisher_result != array('') and finisher_result != array(null)")
    cleaned_df = cleaned_df.dropDuplicates(["finisher_result","label"])

    # remove rows with same comments but different labels
    cleaned_df_2 = cleaned_df.groupby("finisher_result").agg(count("finisher_result").alias("polletti")).filter(col("polletti")>1)
    cleaned_df = cleaned_df.join(cleaned_df_2, ["finisher_result"], 'left_anti')

    cleaned_df_2.unpersist()
    del cleaned_df_2

    print("Dataframe schema:")
    cleaned_df.printSchema()

    cleaned_df.show(10)

    #########################################

    print("Let's verify our dataset is still balanced:")
    cleaned_df.groupBy('label').count().show()

    return cleaned_df

## **Feature Engineering**

### **TF-IDF + PCA**

*Term frequency-inverse document frequency* (TF-IDF) is a feature vectorization method widely used in text mining to reflect the importance of a term to a document in the corpus.
The tf-idf value increases proportionally to the number of times a word appears in the document and is offset by the frequency of the word in the corpus, which helps to adjust for the fact that some words appear more frequently in general.

*Principal component analysis* (PCA) is a dimensionality reduction technique, which is a process of reducing the number of variables under consideration.
More specifically, it is a statistical method to find a rotation such that the first coordinate has the largest variance possible, and each succeeding coordinate, in turn, has the largest variance possible. The columns of the rotation matrix are called principal components.\
We use it to project the vector of extracted features into a low-dimensional space, thus selecting only the most important ones.


In [ ]:
from pyspark.ml.feature import HashingTF, CountVectorizer, IDF
from pyspark.ml import Pipeline, PipelineModel

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import PCA

In [ ]:
def tfidf_pca_features(cleaned_df, column_name="finisher_result", save_parquet=False, load_pipeline=False):

    if load_pipeline:
        features_pca_fitted = PipelineModel.load("PCA_pipeline")
    else:

        ### TF-IDF ###
        hashingTF = HashingTF(inputCol=column_name, outputCol="tf_features", numFeatures=10_000)
        idf = IDF(inputCol="tf_features", outputCol="idf_features")

        ### StandardScaler ###
        scaler = StandardScaler(inputCol="idf_features",
                              outputCol="std_features",
                              withStd=True, withMean=True)

        ### PCA ###
        pca_model = PCA(k=500, inputCol="std_features", outputCol="pca_features")

        tf_idf_pca_pipeline = Pipeline(stages=[hashingTF, idf, scaler, pca_model])
        features_pca_fitted = tf_idf_pca_pipeline.fit(cleaned_df)

        features_pca_fitted.write().overwrite().save("PCA_pipeline")


    features_pca = features_pca_fitted.transform(cleaned_df) \
                          .select(col("comment"), col("pca_features").alias("features"), col("label")) \
                          .withColumn("features", vector_to_array("features")) \
                          .select(['comment'] + ['label'] + [expr('features[' + str(x) + ']') for x in range(500)])

    # cast each column to float
    for col_name in features_pca.columns:
        if col_name == "comment":
            continue
        features_pca = features_pca.withColumn(col_name, col(col_name).cast(FloatType()))

    ### VectorAssembler ###
    vector_assembler = VectorAssembler(inputCols=features_pca.columns[2:], outputCol='features').setHandleInvalid("skip")
    features_pca = vector_assembler.transform(features_pca).select('comment', 'features', 'label')

    print("Dataframe schema:")
    features_pca.printSchema()
    features_pca.show(10)

    # garbage collector
    gc.collect()

    if save_parquet:
        features_pca.write.mode("overwrite").parquet("pca_features.parquet")

    return features_pca

### **Word2Vec**

Word2Vec is a technique that uses a (shallow) neural-network-based model to map each word to a vector of numbers.

In [ ]:
from sparknlp.annotator import Word2VecModel, SentenceEmbeddings, Word2VecApproach
from sparknlp import EmbeddingsFinisher
from pyspark.ml.functions import vector_to_array

In [ ]:
def word2vec_embedding(cleaned_df, pre_trained = False, save_parquet=True):

    df_word2vec = cleaned_df.select('document', 'clean_lemma', 'label', 'comment')

    cleaned_df.unpersist()
    del cleaned_df

    if pre_trained:
        embeddings = Word2VecModel.pretrained("word2vec_gigaword_wiki_300", "en") \
            .setInputCols(["clean_lemma"]) \
            .setOutputCol("embeddings")
    else:
        embeddings = Word2VecApproach() \
            .setInputCols(["clean_lemma"]) \
            .setOutputCol("embeddings") \
            .setMaxIter(5) \
            .setVectorSize(300)

    # the result of w2v are multiple vectors, we will collapse them into one using the average pooling strategy
    embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

    embeddingsFinisher = EmbeddingsFinisher() \
    .setInputCols("sentence_embeddings") \
    .setOutputCols("embeddings_result") \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

    word2vec_pipeline = Pipeline(stages=[embeddings, embeddingsSentence, embeddingsFinisher])

    df_word2vec = word2vec_pipeline.fit(df_word2vec).transform(df_word2vec) \
                                 .select(col("comment"), col('label'), explode("embeddings_result").alias('features')) \
                                 .withColumn("features", vector_to_array("features")) \
                                 .select(['comment'] + ['label'] + [expr('features[' + str(x) + ']') for x in range(300)])

    # cast each column to float
    for col_name in df_word2vec.columns:
        if col_name == "comment":
            continue
        df_word2vec = df_word2vec.withColumn(col_name, col(col_name).cast(FloatType()))

    ### VectorAssembler ###
    vector_assembler = VectorAssembler(inputCols=df_word2vec.columns[2:], outputCol='features').setHandleInvalid("skip")

    features_assembled = vector_assembler.transform(df_word2vec).select('comment', 'features', 'label')

    df_word2vec.unpersist()
    del df_word2vec

    print("Dataframe schema:")
    features_assembled.printSchema()
    features_assembled.show(10)

    if save_parquet:
        features_assembled.write.mode("overwrite").parquet("word2vec_features.parquet")

    # garbage collector
    gc.collect()

    return features_assembled

### **BERT**

*Bidirectional Encoder Representations from Transformers* (BERT) is a state-of-the-art transformer-based model for natural language processing (NLP). It is designed to capture contextual information from text by using a deep bidirectional architecture.
BERT generates contextualized word embeddings, known as BERT embeddings, which are pre-trained on large amounts of unlabeled text and then fine-tuned for specific NLP tasks.

The BERT models used in this notebook provides a **word-level embedding** using the BERT architecture. They take as input a sequence of tokens and generate contextualized embeddings for each token in the sequence. The models capture the context of each token based on its surrounding tokens in the sequence.

In [ ]:
from sparknlp.annotator import BertEmbeddings, SentenceEmbeddings
from sparknlp import EmbeddingsFinisher

In [ ]:
def bert_embedding(cleaned_df, save_parquet=False, model="small_bert_L2_768"):
    df_bert = cleaned_df.select('document', 'clean_lemma', 'label', 'comment')

    cleaned_df.unpersist()
    del cleaned_df

    # bert_base_uncased -> the model is imported from https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1
    # small_bert_L2_768 -> the model is imported from https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1
    embeddings = BertEmbeddings.pretrained(model, 'en') \
                .setInputCols("document", "clean_lemma") \
                .setOutputCol("embeddings")
    
    # the result of BERT are multiple vectors, we will collapse them into one using the average pooling strategy
    embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

    finisher = EmbeddingsFinisher() \
              .setInputCols("sentence_embeddings") \
              .setOutputCols("embeddings_result")

    bert_pipeline = Pipeline(stages=[embeddings, embeddingsSentence, finisher])

    df_bert = bert_pipeline.fit(df_bert).transform(df_bert) \
                         .select(col("comment"), col('label').cast(FloatType()), explode("embeddings_result").alias('features')) \
                         .select(['comment'] + ['label'] + [expr('features[' + str(x) + ']') for x in range(768)])

    ### VectorAssembler ###
    vector_assembler = VectorAssembler(inputCols=df_bert.columns[2:], outputCol='features').setHandleInvalid("skip")

    features_assembled = vector_assembler.transform(df_bert).select('comment', 'features', 'label')
    
    df_bert.unpersist()
    del df_bert

    print("Dataframe schema:")
    features_assembled.printSchema()

    # garbage collector
    gc.collect()

    if save_parquet:
        features_assembled.write.mode("overwrite").parquet("bert_features_768.parquet")

    return features_assembled

### Split dataset
Split original dataset into 2 subdatasets: training and testing.

In [ ]:
def data_split(df_features):
    print("Dataframe size in data_split: {:d} instances".format(df_features.count()))

    # Randomly split our original dataset into 80÷20 for training and test, respectively
    train_df, test_df = df_features.randomSplit([0.8, 0.2], seed = 42)

    df_features.unpersist()
    del df_features

    print("Training set size: {:d} instances".format(train_df.count()))
    print("Test set size: {:d} instances".format(test_df.count()))

    print("\nLet's verify our datasets are still balanced:")
    train_df.groupBy('label').count().show()
    test_df.groupBy('label').count().show()

    return train_df, test_df

## **Classification models**

### **Logistic regression**

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
def logistic_regression_classifier(train_df, test_df, suffix_path="", save_model=False):

    lr = LogisticRegression(featuresCol='features',
                          labelCol='label',
                          maxIter=100,
                          regParam=0.3,
                          elasticNetParam=0.8)

    ### Search for the best model's parameters. ###
    
    # We use a ParamGridBuilder to construct a grid of parameters to search over
    param_grid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.0, 0.05, 0.1]) \
                .addGrid(lr.elasticNetParam, [0.5, 0.8, 1.0]) \
                .build()

    cross_val = CrossValidator(estimator=lr,
                             estimatorParamMaps=param_grid,
                             evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"),
                             numFolds=5,
                             parallelism=5)

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train_df)

    best_lr = cv_model.bestModel

    print("--- Best model's parameters: ---\n- reParam = {:f}\n- elasticNetParam = {:f}".format(best_lr.getRegParam(), best_lr.getElasticNetParam()))

    lr_predictions = best_lr.transform(test_df)
    lr_prediction.show(10)

    if save_model:
        path = "models/logisticRegression_" + suffix_path
        best_lr.write().overwrite().save(path)

    # garbage collector
    gc.collect()

    return lr_predictions

### **Random forest**

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

In [ ]:
def random_forest_classifier(train_df, test_df, suffix_path='',save_model=False):

    rf = RandomForestClassifier(featuresCol='features',
                              labelCol='label')

    ### Search for the best model's parameters. ###

    # We use a ParamGridBuilder to construct a grid of parameters to search over
    param_grid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [3, 5, 8]) \
    .addGrid(rf.numTrees, [5, 10]) \
    .build()

    cross_val = CrossValidator(estimator=rf,
                             estimatorParamMaps=param_grid,
                             evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"),
                             numFolds=5,
                             parallelism=5)

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train_df)

    best_rf = cv_model.bestModel

    print("--- Best model's parameters: ---\n- maxDepth = {:d}\n- numTrees = {:d}".format(best_rf.getMaxDepth(), best_rf.getNumTrees()))

    rf_predictions = best_rf.transform(test_df)
    rf_predictions.show(10)

    if save_model:
        path = "models/randomForest_" + suffix_path
        best_rf.write().overwrite().save(path)

    # garbage collector
    gc.collect()

    return rf_predictions

### **Linear Support Vector Machine**

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

In [ ]:
def linear_svc_classifier(train_df, test_df, suffix_path='', save_model=False):

    lsvc = LinearSVC(featuresCol='features', labelCol='label', regParam=0.1)

    ### Search for the best model's parameters. ###

    # We use a ParamGridBuilder to construct a grid of parameters to search over
    param_grid = ParamGridBuilder() \
                .addGrid(lsvc.regParam, [0.0, 0.05, 0.1]) \
                .addGrid(lsvc.maxIter, [50, 100]) \
                .build()

    cross_val = CrossValidator(estimator=lsvc,
                             estimatorParamMaps=param_grid,
                             evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"),
                             numFolds=5,
                             parallelism=5)

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train_df)

    best_lsvc = cv_model.bestModel

    print("--- Best model's parameters: ---\n- regParam = {:f}\n- maxIter = {:f}".format(best_lsvc.getRegParam(), best_lsvc.getMaxIter()))

    lsvc_predictions = best_lsvc.transform(test_df)
    lsvc_predictions.show(10)

    if save_model:
        path = "models/linearSVC_" + suffix_path
        best_lsvc.write().overwrite().save(path)

    # garbage collector
    gc.collect()

    return lsvc_predictions

### **Multilayer perceptron**

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [ ]:
def mlp_classifier(train_df, test_df, suffix_path='', save_model=False):

    if suffix_path == 'Word2Vec':
        num_features = 300
    elif suffix_path == 'tfidf':
        num_features = 500
    elif suffix_path == 'BERT_uncased' or suffix_path == 'BERT_small':
        num_features = 768
    else:
        num_features = 1

    # specify layers for the neural network:
    input_layer = num_features # features
    output_layer = 2 # number of classes
    layers = [input_layer, 64, 32, output_layer]

    mlp = MultilayerPerceptronClassifier(featuresCol='features', labelCol='label',
                                       maxIter=100, layers=layers, blockSize=256, seed=42)

    # Run cross-validation, and choose the best set of parameters.
    mlp_model = mlp.fit(train_df)

    print("--- Model's parameters: ---\n- layers = {:s}".format(str(mlp_model.getLayers())))
        
    mlp_predictions = mlp_model.transform(test_df)
    mlp_predictions.show(10)

    if save_model:
        path = "models/mlp_" + suffix_path
        mlp_model.write().overwrite().save(path)

    # garbage collector
    gc.collect()

    return mlp_predictions

## **Evaluation**

In [ ]:
from sklearn.metrics import classification_report, matthews_corrcoef
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
def evaluation(model_predictions):
    print("***** Test Set *****")

    preds_df = model_predictions.select("comment", "label", "prediction").toPandas()

    print("\nShow some examples of miss-predictions: ")
    display(preds_df[preds_df['prediction'] != preds_df['label']].head(100))

    print("\nClassification report:")
    clf_report = classification_report(y_true=preds_df['label'], y_pred=preds_df['prediction'], zero_division=0)
    print(clf_report)

    evaluator = BinaryClassificationEvaluator()
    evaluator.setRawPredictionCol('rawPrediction')

    # calculate AUROC
    print('Area Under ROC Curve (AUROC): %.3f' % evaluator.evaluate(model_predictions, {evaluator.metricName: "areaUnderROC"}))

    # calculate AUPR
    print('Area Under Precision-Recall Curve (AUPR): %.3f' % evaluator.evaluate(model_predictions, {evaluator.metricName: "areaUnderPR"}))

    # calculate MCC
    print("Matthews Correlation Coefficient (MCC): ", matthews_corrcoef(preds_df['label'], preds_df['prediction']))

    print("\n***** Test Set *****")

## **Experiments**

### Data loading and pre-processing

In [ ]:
df = load_dataset_partitioned(partitioned=True)

In [ ]:
cleaned_df = clean_text(df)

In [ ]:
df.unpersist()
del df

print("Garbage collector: collected %d objects" % (gc.collect()))

### Models + TF-IDF + PCA

In [ ]:
# features_df_tfidf = spark.read.parquet("tfidf_features.parquet")
features_df_tfidf = tfidf_pca_features(cleaned_df, save_parquet=False)

In [ ]:
cleaned_df.unpersist()
del cleaned_df

In [ ]:
train_df_tfidf, test_df_tfidf = data_split(features_df_tfidf)

In [ ]:
features_df_tfidf.unpersist()
del features_df_tfidf

In [ ]:
model_predictions = logistic_regression_classifier(train_df_tfidf, test_df_tfidf, "tfidf")
evaluation(model_predictions)

In [ ]:
model_predictions = random_forest_classifier(train_df_tfidf, test_df_tfidf, "tfidf")
evaluation(model_predictions)

In [ ]:
model_predictions = linear_svc_classifier(train_df_tfidf, test_df_tfidf, "tfidf")
evaluation(model_predictions)

In [ ]:
model_predictions = mlp_classifier(train_df_tfidf, test_df_tfidf, "tfidf")
evaluation(model_predictions)

In [ ]:
train_df_tfidf.unpersist()
test_df_tfidf.unpersist()

del train_df_tfidf
del test_df_tfidf

### Models + Word2Vec

In [ ]:
# features_df_word2vec = spark.read.parquet("word2vec_features.parquet")
features_df_word2vec = word2vec_embedding(cleaned_df, save_parquet=False)

In [ ]:
cleaned_df.unpersist()
del cleaned_df

In [ ]:
train_df_word2Vec, test_df_word2Vec = data_split(features_df_word2vec)

In [ ]:
features_df_word2vec.unpersist()
del features_df_word2vec

In [ ]:
model_predictions = logistic_regression_classifier(train_df_word2Vec, test_df_word2Vec, "Word2Vec")
evaluation(model_predictions)

In [ ]:
model_predictions = random_forest_classifier(train_df_word2Vec, test_df_word2Vec, "Word2Vec")
evaluation(model_predictions)

In [ ]:
model_predictions = mlp_classifier(train_df_word2Vec, test_df_word2Vec, "Word2Vec")
evaluation(model_predictions)

In [ ]:
model_predictions = linear_svc_classifier(train_df_word2Vec, test_df_word2Vec, "Word2Vec")
evaluation(model_predictions)

In [ ]:
train_df_word2Vec.unpersist()
test_df_word2vec.unpersist()

del train_df_word2Vec
del test_df_word2vec

### Models + Word2Vec (pre-trained)

In [ ]:
features_df_word2vec = word2vec_embedding(cleaned_df, pre_trained = True, save_parquet=False)

In [ ]:
cleaned_df.unpersist()
del cleaned_df

In [ ]:
train_df_word2Vec, test_df_word2Vec = data_split(features_df_word2vec)

In [ ]:
features_df_word2vec.unpersist()
del features_df_word2vec

In [ ]:
model_predictions = logistic_regression_classifier(train_df_word2Vec, test_df_word2vec, "Word2Vec")
evaluation(model_predictions)

In [ ]:
model_predictions = random_forest_classifier(train_df_word2Vec, test_df_word2vec, "Word2Vec")
evaluation(model_predictions)

In [ ]:
model_predictions = linear_svc_classifier(train_df_word2Vec, test_df_word2vec, "Word2Vec")
evaluation(model_predictions)

In [ ]:
model_predictions = mlp_classifier(train_df_word2Vec, test_df_word2vec, "Word2Vec")
evaluation(model_predictions)

In [ ]:
train_df_word2Vec.unpersist()
test_df_word2vec.unpersist()

del train_df_word2Vec
del test_df_word2vec

### Models + BERT (small_bert_L2_768)

In [ ]:
# features_df_bert = spark.read.parquet("bert_features.parquet")
features_df_bert = bert_embedding(cleaned_df, save_parquet=False)

In [ ]:
cleaned_df.unpersist()
del cleaned_df

In [ ]:
train_df_bert, test_df_bert = data_split(features_df_bert)

In [ ]:
features_df_bert.unpersist()
del features_df_bert
gc.collect()

In [ ]:
model_predictions = logistic_regression_classifier(train_df_bert, test_df_bert, "BERT_small")
evaluation(model_predictions)

In [ ]:
model_predictions = random_forest_classifier(train_df_bert, test_df_bert, "BERT_small")
evaluation(model_predictions)

In [ ]:
model_predictions = linear_svc_classifier(train_df_bert, test_df_bert, "BERT_small")
evaluation(model_predictions)

In [ ]:
model_predictions = mlp_classifier(train_df_bert, test_df_bert, "BERT_small")
evaluation(model_predictions)

In [ ]:
train_df_bert.unpersist()
test_df_bert.unpersist()
model_predictions.unpersist()

del train_df_bert
del test_df_bert
del model_predictions

### Models + BERT (bert_base_uncased)

In [ ]:
# features_df_bert = spark.read.parquet("bert_features.parquet")
features_df_bert = bert_embedding(cleaned_df, save_parquet=False, model="bert_base_uncased")

In [ ]:
cleaned_df.unpersist()
del cleaned_df

In [ ]:
train_df_bert, test_df_bert = data_split(features_df_bert)

In [ ]:
features_df_bert.unpersist()
del features_df_bert

gc.collect()

In [ ]:
model_predictions = logistic_regression_classifier(train_df_bert, test_df_bert, "BERT_uncased")
evaluation(model_predictions)

In [ ]:
model_predictions = random_forest_classifier(train_df_bert, test_df_bert, "BERT_uncased")
evaluation(model_predictions)

In [ ]:
model_predictions = linear_svc_classifier(train_df_bert, test_df_bert, "BERT_uncased")
evaluation(model_predictions)

In [ ]:
model_predictions = mlp_classifier(train_df_bert, test_df_bert, "BERT_uncased")
evaluation(model_predictions)

In [ ]:
train_df_bert.unpersist()
test_df_bert.unpersist()
model_predictions.unpersist()

del train_df_bert
del test_df_bert
del model_predictions